In [94]:
# # Make sure to run this cell to use torchmetrics.
!pip install torch torchvision torchmetrics


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [95]:
# Set random seeds for reproducibility
import random
import numpy as np
import torch
from torchvision import models
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(101010)
np.random.seed(101010)
random.seed(101010)

In [96]:
import os
import zipfile

# Unzip the data folder
if not os.path.exists('data/chestxrays'):
    with zipfile.ZipFile('data/chestxrays.zip', 'r') as zip_ref:
        zip_ref.extractall('data')

In [97]:
# Data loading
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

# Define transformers
transform_mean = [0.485, 0.456, 0.406]
transform_std = [0.229, 0.224, 0.225]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize(mean=transform_mean, std=transform_std)])

# Create dataset
train_dataset = ImageFolder('data/chestxrays/train', transform=transform)
test_dataset = ImageFolder('data/chestxrays/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, 
                          batch_size=40, 
                          shuffle=True)
test_loader = DataLoader(test_dataset, 
                         batch_size=len(test_dataset))

In [98]:
# Train model 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.base = models.resnet18(pretrained=True)
        self.fc1 = nn.Linear(1000, 512)
        self.fc2 = nn.Linear(512, 1)
        
        # Freeze all layers except the last fully connected layer of the base model
        for param in self.base.parameters():
            param.requires_grad = False
        for param in self.base.fc.parameters():
            param.requires_grad = True

    def forward(self, x):
        x = self.base(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [104]:
import torch.optim as optim
net = Net()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

for epoch in range(5): 
    
    # Initialize the running loss and accuracy
    running_loss = 0.0
    running_accuracy = 0
    
    for features, labels in train_loader:
        optimizer.zero_grad()
                
        labels = labels.float().unsqueeze(1)
        outputs = net(features) 
        preds = torch.sigmoid(outputs).round()  # Round to 0 or 1

        loss = criterion(outputs, labels)  # Ensure labels are of type float
        loss.backward()
        optimizer.step()
        
        # Update the running loss and accuracy
        running_loss += loss.item() * inputs.size(0)
        running_accuracy += torch.sum(preds == labels.data)

    # Calculate the train loss and accuracy for the current epoch
    train_loss = running_loss / len(train_dataset)
    train_acc = running_accuracy.double() / len(train_dataset)

    # Print the epoch results
    print('Epoch [{}/{}], train loss: {:.4f}, train acc: {:.4f}'
              .format(epoch+1, 5, train_loss, train_acc))

Epoch [1/5], train loss: 1.4968, train acc: 0.7267
Epoch [2/5], train loss: 1.0904, train acc: 0.8533
Epoch [3/5], train loss: 0.8910, train acc: 0.8900
Epoch [4/5], train loss: 0.7544, train acc: 0.8967
Epoch [5/5], train loss: 0.7527, train acc: 0.8967


In [105]:
# Evaluate model
from torchmetrics import Accuracy, F1Score

# Set model to evaluation mode
model = net
model.eval()

# Initialize metrics for accuracy and F1 score
accuracy_metric = Accuracy(task="binary")
f1_metric = F1Score(task="binary")

# Create lists store all predictions and labels
all_preds = []
all_labels = []

# Disable gradient calculation for evaluation
with torch.no_grad():
  for inputs, labels in test_loader:
    # Forward pass
    outputs = model(inputs)
    preds = torch.sigmoid(outputs).round()  # Round to 0 or 1

    # Extend the lists with predictions and labels
    all_preds.extend(preds.tolist())
    all_labels.extend(labels.unsqueeze(1).tolist())

    # Convert lists back to tensors
    all_preds = torch.tensor(all_preds)
    all_labels = torch.tensor(all_labels)

    # Calculate accuracy and F1 score
    test_acc = accuracy_metric(all_preds, all_labels).item()
    test_f1 = f1_metric(all_preds, all_labels).item()

In [108]:
test_acc

0.6200000047683716